In [ ]:
import pandas as pd

df = pd.read_csv("../datasets/train_df.csv")

df

Regexp Tokenizer

In [ ]:
from nltk.tokenize import RegexpTokenizer


def regex_word_tokenizer(sent):
    tokenizer = RegexpTokenizer(r'\w+')

    sample_word_tokens = tokenizer.tokenize(sent)
    sample_word_tokens = [word.lower() for word in sample_word_tokens]
    return sample_word_tokens

words = regex_word_tokenizer(str("Life is beautiful so Enjoy everymoment you have. Runners run hard to win"))
words

Stopwords removal

In [ ]:
from nltk.corpus import stopwords

print(stopwords.words('english'))

Lemmatization

In [ ]:
def stop_words_removal(words):
    stop_words = [word.lower() for word in stopwords.words('english')]
    word_tokens = [word for word in words if word.lower() not in stop_words]
    return word_tokens

stop_words_removal(words)

In [ ]:
from nltk.stem import WordNetLemmatizer

def Lemmatizer(words):
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return lemmatized_words

Lemmatizer(stop_words_removal(words))

In [ ]:
def format_sentence(sent):
    
    # sentence = spellchecker(sent)
    # tokens = word_tokenize(str(sent))
    tokens = regex_word_tokenizer(str(sent))
    # tokens = stop_words_removal(tokens)
    # tokens = Lemmatizer(tokens)

    sentence = ""
    for word in tokens:
        sentence += word + " "
    return sentence

format_sentence("Life havv prpblems beautiful so Enjoy every moment you have. Runners run hard to win caring careful careless")

In [ ]:
X = []
Y = []


for index,row in df.iterrows(): 
    if(row['target']==1):
        X.append(format_sentence(row['question_text']))
        Y.append(1)
    else:
        X.append(format_sentence(row['question_text']))
        Y.append(0)

X = pd.DataFrame(X,columns=["text"])
Y = pd.DataFrame(Y)
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X["text"],Y,test_size=0.01,random_state = 42)
print(X_train)


In [ ]:
import fasttext as ft

# Load the pretrained model
ft_model = ft.load_model("../pretrained_model/lid.176.bin")

def fasttext_language_predict(text, model = ft_model):

  text = text.replace('\n', " ")
  prediction = model.predict([text])

  return prediction

In [ ]:
X = []
Y = []

en_0 = 0
nen_0 = 0
en_1 = 0
nen_1 = 0

for index,row in df.iterrows():
    if(row['target']==1):
        if fasttext_language_predict(format_sentence(row['question_text']))[0][0][0][9:]=="en" and fasttext_language_predict(format_sentence(row['question_text']))[1][0][0]>0.7:
            en_1+=1
        else:
            if index <=1000:
                print(format_sentence(row['question_text']),1)
            nen_1+=1

    else:
        if fasttext_language_predict(format_sentence(row['question_text']))[0][0][0][9:]=="en" and fasttext_language_predict(format_sentence(row['question_text']))[1][0][0]>0.7:
            en_0+=1
        else:
            if index <=1000:
                print(format_sentence(row['question_text']))
            nen_0+=1

    if(row['target']==1):
        X.append(format_sentence(row['question_text']))
        Y.append(1)
    else:
        # if fasttext_language_predict(format_sentence(row['question_text']))[0][0][0][9:]=="en" and fasttext_language_predict(format_sentence(row['question_text']))[1][0][0]>0.7:
        X.append(format_sentence(row['question_text']))
        Y.append(0)

X = pd.DataFrame(X,columns=["text"])
Y = pd.DataFrame(Y)

print(en_1,nen_1,en_0,nen_0)
X

In [ ]:
# from sklearn.model_selection import train_test_split
# from imblearn.under_sampling import RandomUnderSampler
# import numpy as np
# rus = RandomUnderSampler(random_state=42,sampling_strategy=0.20)
# X_data = X["text"].values.reshape(-1,1)
# X_train, y_train= rus.fit_resample(X_data,Y)

# print(len(X_train),len(y_train))

# ones = 0
# zeros = 0
# y_train = np.array(y_train)
# for i in range(0,len(y_train)):
#     if not y_train[i]:
#         zeros+=1
#     else:
#         ones+=1
# print(ones,zeros)

# X_train,X_test,Y_train,Y_test = train_test_split(X_train,y_train,random_state = 42)
# X_train = X_train.flatten()
# X_test = X_test.flatten()
# Y_train = Y_train.flatten()
# Y_test = Y_test.flatten()
# print(X_train)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 

# tfidf_vectorizer=TfidfVectorizer(use_idf=True,max_df=0.25, sublinear_tf=True)
tfidf_vectorizer=TfidfVectorizer(sublinear_tf=True,ngram_range= (1,3),smooth_idf=False,min_df=0.1)
vect=tfidf_vectorizer.fit_transform(X_train) 
vect=tfidf_vectorizer.fit(X_train)

# vect.get_feature_names()[::1000]


In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer

# vect = CountVectorizer().fit(X_train)

# from sklearn.feature_extraction.text import CountVectorizer

# vect = CountVectorizer().fit(X_train)

##TFIDF
# from sklearn.feature_extraction.text import TfidfVectorizer 
# tfidf_vectorizer=TfidfVectorizer(use_idf=True) 
# vect=tfidf_vectorizer.fit_transform(X_train) 
# vect=tfidf_vectorizer.fit(X_train)

# vect.get_feature_names()[::1000]

In [ ]:
X_train_vectorised = vect.transform(X_train)
X_train_vectorised
# print(len(Y_train[0]))

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

# clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=3, random_state=0)
clf = LogisticRegression(penalty = "l2",max_iter = 1000000,random_state = 42,solver="lbfgs",class_weight={0:1,1:1.11},dual=False,intercept_scaling=1000)

clf.fit(X_train_vectorised,np.array(Y_train[0]))





In [ ]:
# from sklearn.svm import SVC
# import numpy as np
# svclassifier = SVC(kernel='sigmoid')
# svclassifier.fit(X_train_vectorised,np.array(Y_train[0]))
# y_pred = svclassifier.predict(X_test)


In [ ]:
y_pred = np.array([])
probas = clf.predict_proba(vect.transform(X_test))
y_pred = []
for i in range(len(probas)):
  if(probas[i][0]<0.8):
    y_pred.append(1)
  else:
    y_pred.append(0)

In [ ]:
from sklearn.metrics import confusion_matrix

cf_matrix = confusion_matrix(np.array(Y_test[0]), y_pred)
print(cf_matrix)

In [ ]:
import  seaborn as sns

In [ ]:
group_names = ["True Neg","False Pos","False Neg","True Pos"]
group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')

In [ ]:
from sklearn.metrics import f1_score

f1_score(np.array(Y_test[0]),y_pred)

In [ ]:
test_df = pd.read_csv("../datasets/test_df.csv")
test_df

In [ ]:
y_pred = np.array([])
probas = clf.predict_proba(vect.transform(test_df["question_text"]))
y_pred = []
for i in range(len(probas)):
  if(probas[i][0]<0.8):
    y_pred.append(1)
  else:
    y_pred.append(0)

In [ ]:
test_df.drop(["question_text"],axis=1,inplace=True)

In [ ]:
test_df["target"]=y_pred

In [ ]:
test_df.to_csv('../results/lr_withProb0.80_classWeight_withLang_maxdf0.25.csv', index=False)